In [1]:
import cv2
from numpy import *
import numpy as np
import glob

# draw a cube on the img with imgpts
def draw_cube(img, imgpts):
    imgpts = np.int32(imgpts).reshape(-1,2)

    # draw surround layers in blue and green colors
    for i in range(4):
        if i & 1 ==0:
            cv2.drawContours(img, [imgpts[[i,(i+1)%4,(i+1)%4+4,i%4+4]]],-1,(255,0,0),-3)
        else:
            cv2.drawContours(img, [imgpts[[i,(i+1)%4,(i+1)%4+4,i%4+4]]],-1,(0,0,255),-3)

    # draw top layer in red color
    cv2.drawContours(img, [imgpts[4:]],-1,(0,255,0),-3)

    return img

In [ ]:
# convert image points to world points
def img2worldpoint(imgpts,rvec,tvec,mtx):

    pts = np.ones((1,3),np.float32)
    pts[0][0]=imgpts[0][0]
    pts[0][1]=imgpts[0][1]
    pts[0][2]=1
    X = mat(pts)

    # convert rotation vector to rotaion matrix
    rmtx, jab = cv2.Rodrigues(rvec)

    R = mat(rmtx).T
    t = mat(tvec)
    K = mat(mtx).T

    Rt = np.ones((3,4),np.float32)
    Rt[:,0]=R[0,:]
    Rt[:,1]=R[1,:]
    Rt[:,2]=R[2,:]
    Rt[:,3]=t.T
    tform = K*Rt
    U = tform.I * X.T
    # U = (K*R).I * (X.T-K*t)

    wpts = np.zeros((3,1),np.float32)
    wpts[0] = U[0]/U[3]
    wpts[1] = U[1]/U[3]
    wpts[2] = U[2]/U[3]


    ori = R.T
    locat = -t.T*ori
    return wpts, locat, R, t, K

In [2]:
# Load previously saved data
with np.load('point.npz') as X:
    # print(X)
    mtx, dist, _, _ = [X[i] for i in ('mtx','dist','tvecs','rvecs')]


In [3]:
# termination criteria
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
weight = 7
height = 5

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((weight*height,3), np.float32)
objp[:,:2] = np.mgrid[0:weight,0:height].T.reshape(-1,2)

# axis = np.float32([[3,0,0], [0,3,0], [0,0,-3]]).reshape(-1,3)

axis = np.float32([[0,0,0], [0,3,0], [3,3,0], [3,0,0],
                   [0,0,-3],[0,3,-3],[3,3,-3],[3,0,-3] ])


In [4]:
cap = cv2.VideoCapture(0)

while(1):
    ret, frame = cap.read()
    cv2.imshow("Camera",frame)
    gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)

    # Find the chess board corners
    ret, corners = cv2.findChessboardCorners(gray, (weight,height),None)
    if ret == True:
        corners2 = cv2.cornerSubPix(gray,corners,(11,11),(-1,-1),criteria)
        
        
        # Find the rotation and translation vectors.
        ret1, rvecs, tvecs, inliers = cv2.solvePnPRansac(objp, corners2, mtx, dist)

        # project 3D points to image plane
        imgpts, jac = cv2.projectPoints(axis, rvecs, tvecs, mtx, dist)

        img = draw(frame,corners2,imgpts)
        cv2.imshow('img',frame)
    if cv2.waitKey(1) & 0xFF ==ord('q'):
        break

cap.release()
cv2.destroyAllWindows()